In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [2]:
calls_df = pd.read_csv('callls.csv')

In [3]:
calls_df.head()

,StartDateTime,UserId,CallType,disconnectType,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time,Unnamed: 7,Unnamed: 8
0,1/1/2023 5:01:42 AM,da7fadb4-cad9-4062-8271-bbc999ac1275,Inbound_Queue,endpoint,1188.162,0.000,1075.162,NaN,NaN
1,1/1/2023 5:11:53 AM,af512203-3ea2-48d5-bbf5-e46ae915d4d8,Inbound_Queue,peer,396.923,0.000,216.923,NaN,NaN
2,1/1/2023 5:24:17 AM,6b633a32-ec20-48f4-91dd-503d459eda1f,Inbound_Queue,peer,537.893,173.363,361.544,NaN,NaN
3,1/1/2023 6:01:03 AM,63bb6f93-004c-4a79-8d01-7de24f75241b,Inbound_Queue,peer,266.625,0.000,263.625,NaN,NaN
4,1/1/2023 6:04:02 AM,e47ca893-1edc-49e9-b0bd-58cde8182443,Inbound_Queue,peer,2545.198,856.591,1674.901,NaN,NaN


In [4]:
calls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135859 entries, 0 to 135858
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   StartDateTime      135859 non-null  object 
 1   UserId             135859 non-null  object 
 2   CallType           135859 non-null  object 
 3   disconnectType     135830 non-null  object 
 4   Total_Handle_Time  135859 non-null  float64
 5   Total_Hold_Time    135859 non-null  float64
 6   Total_Talk_Time    135859 non-null  float64
 7   Unnamed: 7         0 non-null       float64
 8   Unnamed: 8         0 non-null       float64
dtypes: float64(5), object(4)
memory usage: 9.3+ MB


In [5]:
calls_df = calls_df.drop(columns=['Unnamed: 7', 'Unnamed: 8'])

In [6]:
calls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135859 entries, 0 to 135858
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   StartDateTime      135859 non-null  object 
 1   UserId             135859 non-null  object 
 2   CallType           135859 non-null  object 
 3   disconnectType     135830 non-null  object 
 4   Total_Handle_Time  135859 non-null  float64
 5   Total_Hold_Time    135859 non-null  float64
 6   Total_Talk_Time    135859 non-null  float64
dtypes: float64(3), object(4)
memory usage: 7.3+ MB


In [7]:
calls_df['StartDateTime'] = pd.to_datetime(calls_df['StartDateTime'], errors= 'coerce')

In [8]:
calls_df = calls_df.sort_values('StartDateTime').reset_index(drop=True)

In [9]:
Target = 'Total_Handle_Time'

In [10]:
calls_df = calls_df.set_index('StartDateTime')

timeseries = calls_df.resample('10T').agg({
    'Total_Handle_Time': 'mean',
    'Total_Talk_Time': 'mean',
    'Total_Hold_Time': 'mean'
    
})

/tmp/ipykernel_11089/4284140803.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timeseries = calls_df.resample('10T').agg({


In [11]:
timeseries

,Total_Handle_Time,Total_Talk_Time,Total_Hold_Time
StartDateTime,,,
2023-01-01 05:00:00,1188.162000,1075.16200,0.000000
2023-01-01 05:10:00,396.923000,216.92300,0.000000
2023-01-01 05:20:00,537.893000,361.54400,173.363000
2023-01-01 05:30:00,NaN,NaN,NaN
2023-01-01 05:40:00,NaN,NaN,NaN
...,...,...,...
2023-03-15 18:10:00,1558.753167,1003.22800,416.483333
2023-03-15 18:20:00,834.522000,692.76725,23.573500
2023-03-15 18:30:00,1491.483750,1257.56850,53.915250


In [12]:
timeseries = timeseries.dropna()

In [13]:
timeseries

,Total_Handle_Time,Total_Talk_Time,Total_Hold_Time
StartDateTime,,,
2023-01-01 05:00:00,1188.162000,1075.162000,0.000000
2023-01-01 05:10:00,396.923000,216.923000,0.000000
2023-01-01 05:20:00,537.893000,361.544000,173.363000
2023-01-01 06:00:00,1279.849333,981.214667,292.399333
2023-01-01 06:10:00,1486.984000,1060.135500,291.098500
...,...,...,...
2023-03-15 18:10:00,1558.753167,1003.228000,416.483333
2023-03-15 18:20:00,834.522000,692.767250,23.573500
2023-03-15 18:30:00,1491.483750,1257.568500,53.915250


## introducing lags

In [14]:
timeseries['lag_1'] = timeseries[Target].shift(1)
timeseries['lag_2'] = timeseries[Target].shift(2)
timeseries['lag_3'] = timeseries[Target].shift(3)

/tmp/ipykernel_11089/344899965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timeseries['lag_1'] = timeseries[Target].shift(1)
/tmp/ipykernel_11089/344899965.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timeseries['lag_2'] = timeseries[Target].shift(2)
/tmp/ipykernel_11089/344899965.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [15]:
timeseries

,Total_Handle_Time,Total_Talk_Time,Total_Hold_Time,lag_1,lag_2,lag_3
StartDateTime,,,,,,
2023-01-01 05:00:00,1188.162000,1075.162000,0.000000,NaN,NaN,NaN
2023-01-01 05:10:00,396.923000,216.923000,0.000000,1188.162000,NaN,NaN
2023-01-01 05:20:00,537.893000,361.544000,173.363000,396.923000,1188.162000,NaN
2023-01-01 06:00:00,1279.849333,981.214667,292.399333,537.893000,396.923000,1188.162000
2023-01-01 06:10:00,1486.984000,1060.135500,291.098500,1279.849333,537.893000,396.923000
...,...,...,...,...,...,...
2023-03-15 18:10:00,1558.753167,1003.228000,416.483333,1257.826500,1469.373857,1355.773333
2023-03-15 18:20:00,834.522000,692.767250,23.573500,1558.753167,1257.826500,1469.373857
2023-03-15 18:30:00,1491.483750,1257.568500,53.915250,834.522000,1558.753167,1257.826500


In [16]:
timeseries['Rolling_Mean_3'] = timeseries[Target].shift(1).rolling(window=3).mean()
timeseries['Rolling_std_3'] = timeseries[Target].shift(1).rolling(window=3).std()

/tmp/ipykernel_11089/678716445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timeseries['Rolling_Mean_3'] = timeseries[Target].shift(1).rolling(window=3).mean()
/tmp/ipykernel_11089/678716445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timeseries['Rolling_std_3'] = timeseries[Target].shift(1).rolling(window=3).std()


In [17]:
timeseries

,Total_Handle_Time,Total_Talk_Time,Total_Hold_Time,lag_1,lag_2,lag_3,Rolling_Mean_3,Rolling_std_3
StartDateTime,,,,,,,,
2023-01-01 05:00:00,1188.162000,1075.162000,0.000000,NaN,NaN,NaN,NaN,NaN
2023-01-01 05:10:00,396.923000,216.923000,0.000000,1188.162000,NaN,NaN,NaN,NaN
2023-01-01 05:20:00,537.893000,361.544000,173.363000,396.923000,1188.162000,NaN,NaN,NaN
2023-01-01 06:00:00,1279.849333,981.214667,292.399333,537.893000,396.923000,1188.162000,707.659333,422.054789
2023-01-01 06:10:00,1486.984000,1060.135500,291.098500,1279.849333,537.893000,396.923000,738.221778,474.329466
...,...,...,...,...,...,...,...,...
2023-03-15 18:10:00,1558.753167,1003.228000,416.483333,1257.826500,1469.373857,1355.773333,1360.991230,105.870161
2023-03-15 18:20:00,834.522000,692.767250,23.573500,1558.753167,1257.826500,1469.373857,1428.651175,154.541151
2023-03-15 18:30:00,1491.483750,1257.568500,53.915250,834.522000,1558.753167,1257.826500,1217.033889,363.834748


In [18]:
timeseries = timeseries.dropna()

In [19]:
timeseries

,Total_Handle_Time,Total_Talk_Time,Total_Hold_Time,lag_1,lag_2,lag_3,Rolling_Mean_3,Rolling_std_3
StartDateTime,,,,,,,,
2023-01-01 06:00:00,1279.849333,981.214667,292.399333,537.893000,396.923000,1188.162000,707.659333,422.054789
2023-01-01 06:10:00,1486.984000,1060.135500,291.098500,1279.849333,537.893000,396.923000,738.221778,474.329466
2023-01-01 06:20:00,656.413000,476.413000,0.000000,1486.984000,1279.849333,537.893000,1101.575444,499.028673
2023-01-01 06:30:00,1156.589000,587.610000,388.979000,656.413000,1486.984000,1279.849333,1141.082111,432.324303
2023-01-01 06:40:00,806.433000,509.644600,133.788400,1156.589000,656.413000,1486.984000,1099.995333,418.167644
...,...,...,...,...,...,...,...,...
2023-03-15 18:10:00,1558.753167,1003.228000,416.483333,1257.826500,1469.373857,1355.773333,1360.991230,105.870161
2023-03-15 18:20:00,834.522000,692.767250,23.573500,1558.753167,1257.826500,1469.373857,1428.651175,154.541151
2023-03-15 18:30:00,1491.483750,1257.568500,53.915250,834.522000,1558.753167,1257.826500,1217.033889,363.834748


## Splitting

In [20]:
x = [[
    timeseries['lag_1'],
    timeseries['lag_2'],
    timeseries['lag_3'],
    timeseries['Rolling_Mean_3'],
    timeseries['Rolling_std_3']
]]

y = timeseries[Target]

split = int(len(timeseries) * 0.7)

x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]




In [24]:
x_test

[]

Baseline model

In [22]:
LinearReg = LinearRegression()

LinearReg.fit(x_train, y_train)
predictions_lr = LinearReg.predict(x_test)
mean_absolute_error(y_test, predictions_lr)

ValueError: Found array with dim 3, while dim <= 2 is required by LinearRegression.